In [5]:
import os.path as op
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

bids_folder = '/Volumes/mrenkeED/data/ds-dnumrisk'
subjects = [f for f in os.listdir(bids_folder) if f[0:3] == 'sub' and len(f) == 6] # int(f[4:]) 

In [ ]:
/Volumes/mrenkeED/data/ds-dnumrisk/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-1_desc-confounds_timeseries.tsv
/Volumes/mrenkeED/data/ds-dnumrisk/derivatives/fmriprep/sub-10/ses-1/func/sub-10_ses-1_task-magjudge_run-1_desc-confounds_timeseries.tsv

In [ ]:
sub = 'sub-10'

fd = []                    
for run in range(1,7):
        fmriprep_confounds_file = op.join(bids_folder,'derivatives', 'fmriprep', sub, f'ses-{ses}', 'func', f'{sub}_ses-{ses}_task-magjudge_run-{run}_desc-confounds_timeseries.tsv')
        fmriprep_confounds = pd.read_table(fmriprep_confounds_file)[fmriprep_confounds_include] 
        fd.append(fmriprep_confounds.mean())

In [13]:
fmriprep_confounds_include = ['framewise_displacement']
ses=1

df_fd = []
subs = []
errors = []     
for sub in  subjects:
        try:
                fd = []                    
                for run in range(1,7):
                        fmriprep_confounds_file = op.join(bids_folder,'derivatives', 'fmriprep', sub, f'ses-{ses}', 'func', f'{sub}_ses-{ses}_task-magjudge_run-{run}_desc-confounds_timeseries.tsv')
                        fmriprep_confounds = pd.read_table(fmriprep_confounds_file)[fmriprep_confounds_include] 
                        fd.append(fmriprep_confounds.mean())
                df_fd.append(np.mean(fd))
                subs.append(int(sub[4:]))
        except:
                errors.append(sub)
        
print(errors)


['sub-10', 'sub-05', 'sub-04', 'sub-11', 'sub-12', 'sub-38', 'sub-47', 'sub-53', 'sub-62', 'sub-65', 'sub-66']


In [4]:
sub

10